In [113]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.webdriver import WebDriver as ChromeWebDriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
from selenium.webdriver.common.action_chains import ActionChains
import json
from selenium.common.exceptions import TimeoutException
import datetime as dt
import re
import os
from urllib.parse import urlparse

chrome_service = ChromeService("C:\\Users\\VICTUS\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
driver = ChromeWebDriver(service=chrome_service)

# Malaysia login url
# login_url="https://shopee.com.my/buyer/login"

# Taiwan login_url
login_url = "https://shopee.tw/buyer/login"
driver.get(login_url)


# language_button = driver.find_element(By.XPATH, '//*[@id="modal"]/div[1]/div[1]/div/div[3]/div[1]/button')
# language_button.click()

username_field = driver.find_element(By.NAME, 'loginKey')
password_field = driver.find_element(By.NAME, 'password')

username_field.send_keys("amit@zethic.com")
password_field.send_keys("Amit@1234#")

login_button = driver.find_element(By.XPATH, "//*[@id='main']/div/div[2]/div/div/div/div[2]/form/div/div[2]/button")
login_button.click()    

# time.sleep(10)

csv_filename = "store.csv"
json_filename = "store.json"

timestamp_last_scrape = None

In [114]:
curr_dir = os.getcwd()
excel_file_path = os.path.join(curr_dir, 'Shoppee_Ratings.xlsx')
column_name = 'url' 
df = pd.read_excel(excel_file_path)
product_urls = df[column_name].tolist()
product_urls = product_urls[:10]
print(product_urls)

['https://shopee.tw/product/9241878/21870358942', 'https://shopee.tw/product/9241878/12687797847', 'https://shopee.tw/product/9241878/17848374918', 'https://shopee.tw/product/9241878/22315898309', 'https://shopee.tw/product/9241878/11118826864', 'https://shopee.tw/product/9241878/12087756208', 'https://shopee.tw/product/9241878/13041538258', 'https://shopee.tw/product/9241878/532288390', 'https://shopee.tw/product/9241878/532316863', 'https://shopee.tw/product/9241878/9475185728']


In [116]:
def extract_customer_name(element):
    name=""
    try:
        name = element.find_element(By.CLASS_NAME, 'shopee-product-rating__author-name').text
    except Exception as e:
        print(f"An error occurred while extracting name data: {e}")
    return name

def extract_profile_url(element):
    url=""
    try:
        url = element.find_element(By.CLASS_NAME, 'shopee-product-rating__author-name').get_attribute("href")
    except Exception as e:
        print(f"An error occurred while extracting url data: {e}")
    return url

def extract_rating(element):
    rating=""
    try:
        svg_elements = element.find_element(By.CLASS_NAME, 'shopee-product-rating__rating')
        rating = len(svg_elements.find_elements(By.CLASS_NAME, 'icon-rating-solid--active'))
    except Exception as e:
        print(f"An error occurred while extracting rating data: {e}")
    return rating

def extract_time(element):
    time=""
    try:
        time = element.find_element(By.CLASS_NAME, 'shopee-product-rating__time').text
    except Exception as e:
        print(f"An error occurred while extracting time data: {e}")
    return time

def extract_review_data(element):
    review_data = ""
    try:
        container = element.find_element(By.CSS_SELECTOR, '.Rk6V\\+3')
        div_elements = container.find_elements(By.TAG_NAME, "div")
        for div in div_elements:
            text = div.text.strip()
            if text: 
                review_data += text + '\n'
    except NoSuchElementException as e:
        return review_data 
    except Exception as e:
        print(f"An error occurred while extracting review data: {e}")
    return review_data

def extract_image_url(element):
    image_urls = []
    try:
        parent_element = element.find_element(By.CLASS_NAME, "rating-media-list__zoomed-image")
        image_elements = parent_element.find_elements(By.TAG_NAME, "img")
        urls = [element.get_attribute("src") for element in image_elements]
        image_urls = urls
    except NoSuchElementException as e:
        return image_urls 
    except Exception as e:
        print(f"An error occurred while extracting image URLs: {e}")
    return image_urls

def extract_video_url(element):
    video_urls = []
    try:
        parent_element = element.find_element(By.CLASS_NAME, "rating-media-list__zoomed-image")
        video_elements = parent_element.find_elements(By.TAG_NAME, "video")
        urls = [element.get_attribute("src") for element in video_elements]
        video_urls = urls
    except NoSuchElementException as e:
        return video_urls 
    except Exception as e:
        print(f"An error occurred while extracting image URLs: {e}")
    return video_urls

def extract_likes(element):
    likes=""
    try:
        likes = element.find_element(By.CLASS_NAME, 'shopee-product-rating__like-count').text
        if likes == "Helpful?":
            likes = 0
    except NoSuchElementException as e:
        return 0 
    except Exception as e:
        print(f"An error occurred while extracting image URLs: {e}")
    return likes


In [125]:
def scrape_reviews(url):
    reviews = []
    global timestamp_last_scrape

    try:
        driver.get(url)

        scroll_count = 5
        for _ in range(scroll_count):
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(2)

        product_name = driver.find_element(By.CLASS_NAME, "_44qnta").text

        current_page = 1  
        while True:
            parent_elements=driver.find_elements(By.CLASS_NAME, "shopee-product-rating__main")
            
            if parent_elements:
                for i in range(len(parent_elements)):
                    customer_name = extract_customer_name(parent_elements[i])
                    customer_profile_url = extract_profile_url(parent_elements[i])
                    rating = extract_rating(parent_elements[i])
                    timeStamp=extract_time(parent_elements[i])
                    review = extract_review_data(parent_elements[i])
                    video_urls=extract_video_url(parent_elements[i])
                    image_urls=extract_image_url(parent_elements[i])
                    media_urls=video_urls+image_urls
                    likes = extract_likes(parent_elements[i])
                    reviews.append([product_name, url, "Historical", customer_name, customer_profile_url, rating, timeStamp, review, media_urls, likes])

            try:
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'shopee-icon-button--right'))
                )
                ActionChains(driver).move_to_element(next_page_button).perform()
                next_page_button.click()
                time.sleep(2)
                new_page = driver.find_element(By.CLASS_NAME, 'shopee-button-solid.shopee-button-solid--primary').text
                if new_page == str(current_page):
                    break
                current_page = new_page
            except TimeoutException:
                print("Pagination button is not clickable. Exiting pagination.")
                break
            except Exception as e:
                print(f"An error occurred while navigating to the next page: {e}")
                break
            
    except Exception as e:
        print(f"An error occurred: {e}")

    return reviews



In [124]:
def create_folder(url):
    a = urlparse(url)
    folder_name = os.path.basename(a.path)
    folder_name = folder_name[:10]
    path = os.path.join(os.getcwd(), "scraped_data", folder_name)
    try: 
        os.makedirs(path, exist_ok=True) 
        print("Directory '%s' created successfully" % folder_name) 
    except OSError as error: 
        print("Directory '%s' can not be created" % folder_name) 
    return path

print(timestamp_last_scrape)
# timestamp_last_scrape = "2023-10-1 18:41"
# timestamp_last_scrape = None


for url in product_urls:
    data=[]
    folder_path = create_folder(url)
    reviews = scrape_reviews(url)

    if timestamp_last_scrape  is None:
        for review in reviews:
            data.append({
                "Radarly pid": 6127,
                "Radarly corpusId": 64143,
                "Radarly corpusName": "Feminine Care",
                "Product_Name": review[0],
                "Product_url": review[1],
                "Status": review[2],
                "Name": review[3],
                "Customer_Profile_Url": review[4],
                "Rating": review[5],
                "TimeStamps": review[6],
                "Review": review[7],
                "Media_Url": review[8],
                "Likes": review[9],
                "Id": ""
            })

        with open(os.path.join(folder_path, "past_reviews.json"), 'w', encoding='utf-8') as past_json_file:
            json.dump(data, past_json_file, ensure_ascii=False, indent=4)

        with open(os.path.join(folder_path, "latest_reviews.json"), 'w', encoding='utf-8') as latest_json_file:
            json.dump(data, latest_json_file, ensure_ascii=False, indent=4)

        print(f"Scraping completed for URL '{url}'. Data saved in '{folder_path}' folder.")
    else:
        temp_timestamp_last_scrape = dt.datetime.strptime(timestamp_last_scrape, '%Y-%m-%d %H:%M')

        with open(os.path.join(folder_path, "past_reviews.json"), 'r', encoding='utf-8') as past_json_file:
            past_reviews_data = json.load(past_json_file)
            
        for review in reviews:
            full_timestamp = review[6]
            timestamp_match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}', full_timestamp)
            if timestamp_match:
                timestamp_part = timestamp_match.group(0)
            else:
                continue
                    
            try:    
                timestamp_part = dt.datetime.strptime(timestamp_part, '%Y-%m-%d %H:%M')
                if timestamp_part > temp_timestamp_last_scrape:
                    data.append({
                        "Radarly pid": 6127,
                        "Radarly corpusId": 64143,
                        "Radarly corpusName": "Feminine Care",
                        "Product_Name": review[0],
                        "Product_url": review[1],
                        "Status": review[2],
                        "Name": review[3],
                        "Customer_Profile_Url": review[4],
                        "Rating": review[5],
                        "TimeStamps": timestamp_part.strftime("%Y-%m-%d %H:%M"),
                        "Review": review[7],
                        "Media_Url": review[8],
                        "Likes": review[9],
                        "Id": ""
                    })
            except ValueError:
                print(f"Error parsing timestamp: {timestamp_part}")
                continue
        
        with open(os.path.join(folder_path, "latest_reviews.json"), 'w', encoding='utf-8') as latest_json_file:
            json.dump(data, latest_json_file, ensure_ascii=False, indent=4)

        print(f"Scraping completed for URL '{url}'. Latest data (not in past_reviews.json) saved in '{folder_path}' folder.")

current_datetime = dt.datetime.now()
timestamp_last_scrape = current_datetime.strftime("%Y-%m-%d %H:%M")



2023-10-11 16:52
Directory '2187035894' created successfully
An error occurred while navigating to the next page: Message: element click intercepted: Element <button class="shopee-icon-button shopee-icon-button--right ">...</button> is not clickable at point (826, 886). Other element would receive the click: <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 20 20" class="chat-icon">...</svg>
  (Session info: chrome=117.0.5938.150)
Stacktrace:
	GetHandleVerifier [0x00007FF6F2BC7D12+55474]
	(No symbol) [0x00007FF6F2B377C2]
	(No symbol) [0x00007FF6F29EE0EB]
	(No symbol) [0x00007FF6F2A34E7B]
	(No symbol) [0x00007FF6F2A33249]
	(No symbol) [0x00007FF6F2A31018]
	(No symbol) [0x00007FF6F2A300D3]
	(No symbol) [0x00007FF6F2A25DCF]
	(No symbol) [0x00007FF6F2A4F15A]
	(No symbol) [0x00007FF6F2A256E6]
	(No symbol) [0x00007FF6F2A4F370]
	(No symbol) [0x00007FF6F2A67EF2]
	(No symbol) [0x00007FF6F2A4EF33]
	(No symbol) [0x00007FF6F2A23D41]
	(No symbol) [0x00007FF6F2A24F84]
	GetHandleVerifier [0x00007F

KeyboardInterrupt: 

In [ ]:
# To store in csv file

with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Radarly pid", "Radarly corpusId", "Radarly corpusName", "Product_Name", "Product_url",  "Status", "Name", "Customer_Profile_Url", "Rating", "TimeStamps", "Review", "Media_Url", "Likes", "Id"])
    
    for url in product_urls:
        historical_reviews = scrape_reviews(url)

        for review in historical_reviews:
            csv_writer.writerow([6127, 64143, "Feminine Care", review[0], review[1], review[2], review[3], review[4], review[5], review[6], review[7], review[8], review[9], ""])

print(f"Scraping completed. Data saved in '{csv_filename}'.")

df=pd.read_csv("store.csv")
df

